In [ ]:
import pandas as pd
import zipfile
import os
import sys
import numpy as np

train_file = 'data/train.csv'
train_df = pd.read_csv(train_file)
test_file = 'data/test.csv'
test_df = pd.read_csv(test_file)
test_df.head()


,mutant
0,V1D
1,V1Y
2,V1C
3,V1A
4,V1E


In [17]:
# query1 and query 2 method
test_df['pos'] = test_df['mutant'].str[1:-1].astype(int)
test_df['wt'] = test_df['mutant'].str[0]
test_df['mut'] = test_df['mutant'].str[-1]

query_mutants = test_df.groupby('pos').apply(lambda x: x.sample(n=1, random_state=42))
query_mutants = query_mutants['mutant'].tolist()
print(len(query_mutants))

596


/var/folders/z2/_0c1fw_d62gfdmm0hfw2lfpr0000gn/T/ipykernel_55540/3233668334.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  query_mutants = test_df.groupby('pos').apply(lambda x: x.sample(n=1, random_state=42))


In [ ]:
# for query 1, select first 100 mutants
query1 = []
query1 = query_mutants[:100]

with open('query1.txt', 'w') as f:
    for mutant in query1:
        f.write(mutant + '\n')

In [ ]:
# for query 2, select last 100 mutants
query2 = []
query2 = query_mutants[-100:]

with open('query2.txt', 'w') as f:
    for mutant in query_mutants:
        f.write(mutant + '\n')